In [1]:
from gru_torch import (
    GRU,
    SequenceDataset,
    create_sliding_window,
    arithmetic_progression,
    mape,
    DataLoader,
    torch
)
import plotly.graph_objects as go
import numpy as np

In [8]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence[:-n])        
    # scaled = sequence[:-n]  
    X, y = create_sliding_window(scaled, window_size, n)
    model = GRU(window_size, hidden_size, n, num_layers=1,
                dropout=0, learning_rate=lr, num_epochs=max_epochs)
    train_loader = DataLoader(dataset=SequenceDataset(X, y),
                              batch_size=batch_size,
                              shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train_model(train_loader, device, verbosity, use_adam)
    train_pred = model.predict(train_loader, device).numpy()
    training_mape = mape(y, train_pred)           
    x_last = np.expand_dims(scaled[-window_size:], axis=0)
    train_loader = DataLoader(dataset=SequenceDataset(x_last, np.expand_dims(sequence[-n:], axis=0)),
                              batch_size=batch_size,
                              shuffle=False)     
    pred = model.predict(train_loader, device).numpy()
    descaled_pred = descale_sequence(pred[-1], mean, deviation)
    # descaled_pred = pred[-1]
    return training_mape, model, descaled_pred, mean, deviation

In [14]:
# Fibonacci sequence generator
def fibonacci_generator(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b


# Squared num sequence generator
def squared_generator(n):        
    for i in range(1, n + 1):
        yield i**2


# Arithmetic progression
def arithmetic_progression(n, a0, d):
    for i in range(n):
        yield a0 + i * d


# x = x0 / 2**i sequence generator
def half_generator(n, fst: float):
    num = fst
    for _ in range(n):
        yield num
        num /= 2


# 1/n sequence generator
def one_by_n_generator(n):    
    for i in range(n):
        yield 1 / (i + 1)


# 1, -1, 1, -1, 1,... sequence generator
def plus_one_minus_one_generator(n):    
    for i in range(n):        
        yield 1 if i % 2 == 0 else -1


# 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,... sequence generator
def one_half_generator(n):
    count = 1
    generated = 0
    while generated < n:
        generated += 1
        yield 1
        for _ in range(count):
            if generated >= n:
                break
            generated += 1
            yield 0.5            
        count += 1


In [10]:
experiments_amount = 4
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 8
hidden_size = 6
output_size = 3

verbosity = 100000
lr = 1e-2
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!


In [11]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [30]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 3
batch_size = 8
output_size = 3

verbosity = 100000
lr = 5e-3
max_epochs = 500
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)        
        
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            sequence, window_size, batch_size,
            size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!


In [31]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [32]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 20
window_size = 3
batch_size = 8
hidden_size = 10
output_size = 3
max_epochs = 2000
verbosity = 100000
lr = 1e-2

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length + output_size)
    print(sequence)   

    mapes_sum = 0
    for _ in range(experiments_amount):
        all_sequences.append(seq_name)        
        
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711]
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529]
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
Обучение завершено!
[1.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.125, 0.1111111111111111, 0.1, 0.09090909090909091, 0.08333333333333333, 0.07692307692307693, 0.07142857142857142, 0.06666666666666667, 0.0625, 0.058823529411764705, 0.05555555555555555, 0.05263157894736842, 0.05, 0.047619047619047616, 0.045454545454545456, 0.043478260869565216]
Обучение завершено!
Обучение завершено!
Об

In [33]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()